## Cookie Factory Optimization Problem
Alex Szpakiewicz, Sara Thibierge, Léonard Roussard

In [41]:
# Library imports
import pandas as pd
import numpy as np

### Data Loading

In [42]:
# Load the defects data from the provided CSV file
defects_file_path = 'defects.csv'
defects_data = pd.read_csv(defects_file_path)

In [43]:
# Set the biscuit dict with the biscuit names as keys and the biscuit properties as values
biscuits = {
    'Biscuit_0': {'length': 4, 'value': 6, 'defect_thresholds': {'a': 4, 'b': 2, 'c': 3}},
    'Biscuit_1': {'length': 8, 'value': 12, 'defect_thresholds': {'a': 5, 'b': 4, 'c': 4}},
    'Biscuit_2': {'length': 2, 'value': 1, 'defect_thresholds': {'a': 1, 'b': 2, 'c': 1}},
    'Biscuit_3': {'length': 5, 'value': 8, 'defect_thresholds': {'a': 2, 'b': 3, 'c': 2}}
}

# Set the dough length
dough_length = 500

## Util functions

In [44]:
def is_valid_solution(solution, biscuits, dough_length, defects):
    """
    Check if a biscuit placement solution is valid
    :param solution: 
    :param biscuits: 
    :param dough_length: 
    :param defects: 
    :return: True or False
    """
    current_position = 0
    for biscuit_name, start_position in solution:
        # Check if the biscuit is placed at an integer position and in the correct order
        if start_position < current_position or start_position >= dough_length:
            return False

        biscuit = biscuits[biscuit_name]
        
        # Check for overlap
        if start_position + biscuit['length'] > dough_length:
            return False

        # Check for defects
        relevant_defects = defects[(defects['x'] >= start_position) & 
                                   (defects['x'] < start_position + biscuit['length'])]
        for defect_class in biscuit['defect_thresholds']:
            max_allowed = biscuit['defect_thresholds'][defect_class]
            defects_count = relevant_defects[relevant_defects['class'] == defect_class].shape[0]
            if defects_count > max_allowed:
                return False

        current_position = start_position + biscuit['length']

    # Check if the total length of biscuits does not exceed the dough length
    if current_position > dough_length:
        return False

    return True

In [ ]:
def can_place_biscuit(biscuit, position, defects):
    """ 
    Check if a biscuit can be placed at a given position considering defect thresholds
    :param biscuit: 
    :param position: 
    :param defects: 
    :return: 
    """
    if position + biscuit['length'] > dough_length:
        return False 

    # Filter defects that fall within the biscuit's range
    relevant_defects = defects[(defects['x'] >= position) & (defects['x'] < position + biscuit['length'])]
    
    # Check if the biscuit's defect thresholds are exceeded
    for defect_class in biscuit['defect_thresholds']:
        max_allowed = biscuit['defect_thresholds'][defect_class]
        defects_count = relevant_defects[relevant_defects['class'] == defect_class].shape[0]
        if defects_count > max_allowed:
            return False
    
    return True

## Genetic Algorithm

In [45]:
import random
from deap import creator, base, tools, algorithms

# Define the individual and fitness function
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Initialize necessary components for the genetic algorithm
toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 0, len(biscuits) - 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, dough_length)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


def evaluate(individual):
    """
    Fitness function to evaluate each individual
    :param individual: 
    :return: fitness value
    """
    total_value = 0
    position = 0
    while position < dough_length:
        biscuit_idx = individual[position]
        biscuit_name = f'Biscuit_{biscuit_idx}'
        biscuit = biscuits[biscuit_name]
        if can_place_biscuit(biscuit, position, defects_data):
            total_value += biscuit['value']
            position += biscuit['length']
        else:
            position += 1
    return total_value,

# Genetic operators
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(biscuits) - 1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Create initial population
population = toolbox.population(n=100)

# Run the genetic algorithm
ngen = 100  # Number of generations
result = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=ngen, verbose=False)

# Best solution
best_individual = tools.selBest(population, 1)[0]
best_fitness = evaluate(best_individual)[0]

# Get best individual's biscuit placement
biscuit_placement = []
position = 0
while position < dough_length:
    biscuit_idx = best_individual[position]
    biscuit_name = f'Biscuit_{biscuit_idx}'
    biscuit = biscuits[biscuit_name]
    if can_place_biscuit(biscuit, position, defects_data):
        biscuit_placement.append((biscuit_name, position))
        position += biscuit['length']
    else:
        position += 1
        
print(f"Total value of biscuits placed: {best_fitness}")
print(f"Is the solution valid? {is_valid_solution(biscuit_placement, biscuits, dough_length, defects_data)}")
biscuit_placement

/opt/homebrew/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/opt/homebrew/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Total value of biscuits placed: 723
Is the solution valid? True


[('Biscuit_1', 0),
 ('Biscuit_0', 9),
 ('Biscuit_3', 13),
 ('Biscuit_1', 18),
 ('Biscuit_3', 26),
 ('Biscuit_3', 31),
 ('Biscuit_0', 36),
 ('Biscuit_0', 40),
 ('Biscuit_3', 45),
 ('Biscuit_1', 51),
 ('Biscuit_1', 59),
 ('Biscuit_1', 67),
 ('Biscuit_0', 75),
 ('Biscuit_3', 79),
 ('Biscuit_0', 84),
 ('Biscuit_3', 88),
 ('Biscuit_0', 93),
 ('Biscuit_1', 99),
 ('Biscuit_0', 107),
 ('Biscuit_0', 111),
 ('Biscuit_0', 119),
 ('Biscuit_1', 123),
 ('Biscuit_2', 131),
 ('Biscuit_3', 133),
 ('Biscuit_0', 139),
 ('Biscuit_3', 144),
 ('Biscuit_1', 149),
 ('Biscuit_1', 158),
 ('Biscuit_0', 167),
 ('Biscuit_0', 172),
 ('Biscuit_1', 176),
 ('Biscuit_0', 186),
 ('Biscuit_3', 190),
 ('Biscuit_0', 195),
 ('Biscuit_3', 199),
 ('Biscuit_0', 204),
 ('Biscuit_1', 208),
 ('Biscuit_3', 216),
 ('Biscuit_0', 221),
 ('Biscuit_0', 225),
 ('Biscuit_1', 229),
 ('Biscuit_3', 237),
 ('Biscuit_0', 242),
 ('Biscuit_0', 246),
 ('Biscuit_3', 250),
 ('Biscuit_0', 255),
 ('Biscuit_3', 259),
 ('Biscuit_3', 264),
 ('Biscuit_1

## Dynamic Programming

In [46]:
def optimize_biscuit_placement(biscuits, dough_length, defects):
    """
    Optimize the biscuit placement using dynamic programming
    :param biscuits: 
    :param dough_length: 
    :param defects: 
    :return: biscuit_placement, total_value
    """
    max_values = np.zeros(dough_length + 1)
    placement_tracker = [None] * (dough_length + 1)  # Track biscuit placements

    for position in range(dough_length):
        max_values[position + 1] = max(max_values[position + 1], max_values[position])

        for biscuit_name, biscuit in biscuits.items():
            if can_place_biscuit(biscuit, position, defects):
                end_position = position + biscuit['length']
                if end_position <= dough_length:
                    new_value = max_values[position] + biscuit['value']
                    if new_value > max_values[end_position]:
                        max_values[end_position] = new_value
                        placement_tracker[end_position] = (biscuit_name, position, biscuit['value'])

    # Reconstruct the placement of biscuits
    biscuit_placement = []
    current_position = dough_length
    while current_position > 0:
        placement = placement_tracker[current_position]
        if placement:
            biscuit_name, position, value = placement
            biscuit_placement.append((biscuit_name, position))
            current_position = position
        else:
            current_position -= 1

    return list(reversed(biscuit_placement)), max_values[-1]


# Optimizing the biscuit placement
biscuit_placement_details, total_value = optimize_biscuit_placement(biscuits, dough_length, defects_data)
print(f"Total value of biscuits placed: {total_value}")
print(f"Is the solution valid? {is_valid_solution(biscuit_placement_details, biscuits, dough_length, defects_data)}")

# Getting detailed placement of biscuits
biscuit_placement_details

Total value of biscuits placed: 760.0
Is the solution valid? True


[('Biscuit_1', 0),
 ('Biscuit_1', 8),
 ('Biscuit_0', 16),
 ('Biscuit_1', 20),
 ('Biscuit_1', 28),
 ('Biscuit_0', 36),
 ('Biscuit_0', 41),
 ('Biscuit_3', 45),
 ('Biscuit_0', 50),
 ('Biscuit_0', 54),
 ('Biscuit_3', 58),
 ('Biscuit_3', 63),
 ('Biscuit_3', 68),
 ('Biscuit_1', 73),
 ('Biscuit_1', 81),
 ('Biscuit_1', 89),
 ('Biscuit_0', 99),
 ('Biscuit_3', 103),
 ('Biscuit_3', 108),
 ('Biscuit_0', 114),
 ('Biscuit_0', 118),
 ('Biscuit_0', 122),
 ('Biscuit_3', 126),
 ('Biscuit_3', 131),
 ('Biscuit_0', 136),
 ('Biscuit_0', 140),
 ('Biscuit_3', 144),
 ('Biscuit_1', 149),
 ('Biscuit_1', 158),
 ('Biscuit_3', 167),
 ('Biscuit_0', 172),
 ('Biscuit_1', 176),
 ('Biscuit_0', 186),
 ('Biscuit_3', 190),
 ('Biscuit_3', 195),
 ('Biscuit_3', 200),
 ('Biscuit_3', 205),
 ('Biscuit_3', 210),
 ('Biscuit_0', 215),
 ('Biscuit_3', 219),
 ('Biscuit_3', 224),
 ('Biscuit_1', 229),
 ('Biscuit_3', 237),
 ('Biscuit_1', 242),
 ('Biscuit_3', 250),
 ('Biscuit_3', 255),
 ('Biscuit_3', 260),
 ('Biscuit_3', 265),
 ('Biscuit_

## Hill Climbing Local Search

In [49]:
def generate_initial_solution():
    """
    Generate an initial solution
    :return: 
    """
    solution = []
    position = 0
    while position < dough_length:
        biscuit_idx = random.randint(0, len(biscuits) - 1)
        biscuit_name = f'Biscuit_{biscuit_idx}'
        biscuit = biscuits[biscuit_name]
        if can_place_biscuit(biscuit, position, defects_data):
            solution.append(biscuit_idx)
            position += biscuit['length']
        else:
            position += 1
    return solution

def evaluate(solution):
    """
    Evaluate a solution
    :param solution: 
    :return: 
    """
    total_value = 0
    position = 0
    for biscuit_idx in solution:
        biscuit_name = f'Biscuit_{biscuit_idx}'
        biscuit = biscuits[biscuit_name]
        if can_place_biscuit(biscuit, position, defects_data):
            total_value += biscuit['value']
            position += biscuit['length']
        else:
            position += 1
    return total_value

def generate_neighbor(solution):
    """
    Generate a neighbor solution
    :param solution: 
    :return: 
    """
    neighbor = solution[:]
    change_idx = random.randint(0, len(neighbor) - 1)
    neighbor[change_idx] = random.randint(0, len(biscuits) - 1)
    return neighbor

def hill_climbing(initial_solution):
    """
    Hill climbing algorithm
    :param initial_solution: 
    :return: 
    """
    current_solution = initial_solution
    current_value = evaluate(current_solution)
    no_improvement_count = 0

    while no_improvement_count < 100:  # Allow some iterations to find improvement
        neighbor = generate_neighbor(current_solution)
        neighbor_value = evaluate(neighbor)

        if neighbor_value > current_value:
            current_solution = neighbor
            current_value = neighbor_value
            no_improvement_count = 0
        else:
            no_improvement_count += 1

    return current_solution, current_value

# Test the local search algorithm
initial_solution = generate_initial_solution()
best_solution, best_value = hill_climbing(initial_solution)
best_solution, best_value

([1,
  1,
  3,
  0,
  2,
  3,
  3,
  2,
  1,
  1,
  2,
  1,
  0,
  1,
  0,
  2,
  1,
  0,
  3,
  0,
  0,
  0,
  2,
  3,
  1,
  0,
  2,
  2,
  3,
  2,
  0,
  0,
  3,
  0,
  2,
  1,
  3,
  2,
  1,
  2,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  3,
  3,
  1,
  0,
  0,
  2,
  3,
  1,
  2,
  1,
  0,
  3,
  1,
  0,
  2,
  3,
  2,
  3,
  3,
  0,
  2,
  0,
  0,
  2,
  2,
  1,
  2,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  2,
  3,
  3,
  0,
  3,
  1,
  0,
  1,
  2,
  3,
  3,
  3,
  1],
 485)